## Training Materials: ALOS-2 Data Preprocessing and Calibration

This notebook provides a practical guide to preprocessing ALOS-2 data, focusing on speckle noise reduction and radiometric calibration.

**I. Introduction to ALOS-2 Data and Preprocessing**

* **1.1 ALOS-2**
    * ALOS-2 (Advanced Land Observing Satellite-2) is a Japanese Earth observation satellite equipped with an L-band synthetic aperture radar (SAR).  It provides high-resolution imagery regardless of weather conditions, making it valuable for various applications.
    * Preprocessing is crucial for enhancing the quality and interpretability of ALOS-2 data by mitigating inherent noise and ensuring accurate measurements.

* **1.2 Common Preprocessing Steps:**
    * **Speckle filtering:** Speckle noise, a granular interference pattern, is inherent in SAR imagery. Speckle filtering reduces this noise, improving image clarity.
    * **Radiometric calibration:** Calibration converts digital pixel values to physical units (e.g., backscatter), enabling quantitative analysis.

**II. Setting up the Environment**

* **2.1 Installing Necessary Libraries:**

In [ ]:
!pip install rasterio numpy geopandas matplotlib scipy scikit-image

* **2.2 Data Acquisition:**
    * ALOS-2 data can be obtained from the Japan Aerospace Exploration Agency (JAXA).
    * Ensure the data is in GeoTIFF format for compatibility with the provided code.

* **2.3 Loading Data with Rasterio:**

In [ ]:
import rasterio
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from scipy.ndimage import uniform_filter
from skimage.transform import resize
from rasterio.mask import mask

image_path = '/home/jovyan/shared/PCN/ALOS-2/ALOS-2 data/Alos_2_2015_HV.tif'  # Replace with your ALOS-2 image path
aoifile = '/home/jovyan/shared/PCN/ALOS-2/ALOS-2 data/AOI_shapefile.shp'
aoifile = gpd.read_file(aoifile)
with rasterio.open(image_path) as src:
    image_data = src.read(1) # Read the first band
    metadata = src.meta
    out_image, out_transform = mask.mask(src, aoifile.geometry, crop=True)
    out_meta = src.meta.copy()
out_meta.update({"driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform})
clipped_image = out_image[0]

**III. Speckle Filtering**

* **3.1 Introduction to Speckle Filtering Techniques:**
    * Various speckle filters exist, including Lee, Frost, and Refined Lee.  The Lee filter is a commonly used adaptive filter.

* **3.2 Implementing the Lee Filter:**

In [ ]:
def lee_filter(img, size=5):
    img_mean = uniform_filter(img, (size, size))
    img_sqr_mean = uniform_filter(img**2, (size, size))
    img_variance = img_sqr_mean - img_mean**2
    overall_variance = np.var(img)
    img_noise_variance = np.maximum(img_variance - overall_variance, 0)
    weights = img_noise_variance / (img_noise_variance + img_variance)
    img_filtered = img_mean + weights * (img - img_mean)
    return img_filtered

filtered_image = lee_filter(clipped_image)

* **3.3 Visualizing the Results:**

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(clipped_image, cmap='gray')
plt.title('Original Image')
plt.subplot(1, 2, 2)
plt.imshow(filtered_image, cmap='gray')
plt.title('Filtered Image')
plt.show()

**IV. Radiometric Calibration**

* **4.1 ALOS-2 Calibration Formula:**
    * `calibrated_image = (10 * np.log10(image**2)) - 83`
    * This formula converts pixel values to decibels, representing radar backscatter. The constants are specific to ALOS-2.

* **4.2 Applying Calibration:**

In [ ]:
calibrated_image = (10 * np.log10(image**2)) - 83

* **4.3 Visualizing Calibrated Data:**

In [ ]:
plt.figure(figsize=(10,5))
plt.imshow(calibrated_image, cmap='gray')
plt.title('Calibrated Image')
plt.show()

**V. Saving Processed Data**

* **5.1 Writing GeoTIFFs with Rasterio:**

In [ ]:
with rasterio.open('filtered_calibrated_image.tif', 'w', **out_meta) as dest:
    dest.write(calibrated_image.astype(rasterio.float32), 1)

* **5.2 File Naming Conventions:**
    * Use descriptive names including date, sensor, and processing steps.